[Poprzedni rozdzial](rozdzial_09_kod_zrodlowy.ipynb)


# Przegląd kodu źródłowego

Ten notebook to przewodnik po całej strukturze projektu. Na początku znajdziesz krótkie
podsumowanie czasów generowania danych, a potem pełne treści wszystkich plików `.py` wraz
z krótkim opisem ich roli. Kod jest wyświetlany, ale nie jest wykonywany.


In [ ]:
from pathlib import Path
import pickle
import pandas as pd
from IPython.display import display


def _find_data_dir() -> Path:
    here = Path.cwd().resolve()
    for parent in [here, *here.parents]:
        candidate = parent / "data"
        if candidate.is_dir():
            return candidate
    return here / "data"


def _load_pickle(path: Path) -> dict:
    try:
        return pickle.loads(path.read_bytes())
    except Exception as exc:
        return {"_error": f"{type(exc).__name__}: {exc}"}


def _format_seconds(value) -> float | None:
    if value is None:
        return None
    try:
        return float(value)
    except (TypeError, ValueError):
        return None


data_dir = _find_data_dir()
print(f"Podsumowanie czasów generowania (katalog: {data_dir})")
print("-")

timing_paths = sorted(data_dir.glob("*_timing.pkl"))
time_paths = sorted(data_dir.glob("*_time.pkl"))
all_paths = timing_paths + time_paths

rows = []
for path in all_paths:
    payload = _load_pickle(path)
    if payload.get("_error"):
        rows.append({
            "plik": str(path),
            "obiekt": None,
            "czas_s": None,
            "średnia_s": None,
            "uwagi": payload["_error"],
        })
        continue

    if path.name.endswith("_timing.pkl"):
        label = payload.get("output_data_file", path.name)
        elapsed = payload.get("elapsed_seconds")
        avg = payload.get("avg_seconds_per_replication")
        if avg is None:
            avg = payload.get("avg_seconds_per_task")
        notes = payload.get("notes", "-")
    else:
        label = f"I_ref dla C={payload.get('C', '?')}"
        elapsed = payload.get("seconds")
        avg = None
        notes = f"R_ref={payload.get('R_ref', '-')}, chunk={payload.get('chunk_size', '-')}"

    rows.append({
        "plik": str(path),
        "obiekt": str(label),
        "czas_s": _format_seconds(elapsed),
        "średnia_s": _format_seconds(avg),
        "uwagi": notes,
    })

if not rows:
    print("Brak plików z czasami w katalogu data.")
else:
    df = pd.DataFrame(rows)
    display(df)


## Funkcja do wyświetlania kodu

Poniższa funkcja `show_code` jedynie wczytuje plik i pokazuje jego treść.Nie uruchamia kodu zawartego w pliku.


In [6]:
from pathlib import Path


def _find_project_root() -> Path:
    here = Path.cwd().resolve()
    for parent in [here, *here.parents]:
        if (parent / "parameters.py").is_file():
            return parent
    return here


def show_code(path: str) -> None:
    root = _find_project_root()
    file_path = (root / path).resolve()
    text = file_path.read_text(encoding="utf-8")
    print(f"\n### {file_path}\n")
    print(text)


## parameters.py

Definiuje parametry wspólne dla symulacji (rynek, bariery, wartości domyślne).


In [7]:
show_code("parameters.py")



### /media/miplock/OS/Users/mikołaj/Desktop/laby_26/monte_carlo_project_2/parameters.py

import numpy as np

# task parameters
r = 0.05                  # risk-free rate
sigma2 = 0.125            # variance
sigma = np.sqrt(sigma2)   # volatility
mu_star = r - sigma2 / 2  # -0.0125
S0 = 100                  # initial stock price
K = 100                   # strike price
T = 1.0                   # time to maturity
C = np.inf                # upper barrier; use np.inf for brak bariery
# dodatkowe bariery do wizualizacji trajektorii
BARRIERS = (105.0, 120.0)

# simulation parameters
n_steps = 12              # number of time steps
R = 100000                # number of simulations
m = 20                    # number of strata for stratified Monte Carlo

# reproducibility
seed = 12345



## models/

Niskopoziomowe narzędzia do symulacji ścieżek i wartości referencyjnych.


### models/__init__.py

Inicjalizacja pakietu i ewentualne eksporty.


In [ ]:
show_code("models/__init__.py")


### models/simulate_gbm_paths.py

Generuje ścieżki GBM używane w estymatorach Monte Carlo.


In [ ]:
show_code("models/simulate_gbm_paths.py")


### models/black_scholes.py

Wzory Blacka-Scholesa i obliczenia referencyjne.


In [ ]:
show_code("models/black_scholes.py")


## payoffs/

Definicje wypłat używane w wycenach.


### payoffs/__init__.py

Inicjalizacja pakietu i eksporty.


In [ ]:
show_code("payoffs/__init__.py")


### payoffs/payoff_up_and_out_call.py

Definiuje wypłatę dla opcji up-and-out call.


In [ ]:
show_code("payoffs/payoff_up_and_out_call.py")


## methods/

Estymatory Monte Carlo i redukcja wariancji.


### methods/__init__.py

Inicjalizacja pakietu i eksporty estymatorów.


In [ ]:
show_code("methods/__init__.py")


### methods/crude_monte_carlo.py

Podstawowy estymator Monte Carlo dla wyceny opcji.


In [ ]:
show_code("methods/crude_monte_carlo.py")


### methods/antithetic_monte_carlo.py

Estymator z antytetycznymi zmiennymi dla redukcji wariancji.


In [ ]:
show_code("methods/antithetic_monte_carlo.py")


### methods/control_variate.py

Estymator z control variate, korzysta z wartości referencyjnych.


In [ ]:
show_code("methods/control_variate.py")


### methods/stratified_monte_carlo.py

Estymator ze stratyfikacją (alokacja proporcjonalna lub optymalna).


In [ ]:
show_code("methods/stratified_monte_carlo.py")


## scripts/

Punkty wejściowe generujące tabele, replikacje i wykresy.


### scripts/compute_I_ref.py

Liczy wartości referencyjne do diagnostyki redukcji wariancji.


In [ ]:
show_code("scripts/compute_I_ref.py")


### scripts/compare_strata_counts.py

Porównuje liczebności stratum i diagnostykę stratyfikacji.


In [ ]:
show_code("scripts/compare_strata_counts.py")


### scripts/plot_strata_error_hist.py

Rysuje histogram błędów estymatorów stratyfikowanych.


In [ ]:
show_code("scripts/plot_strata_error_hist.py")


### scripts/plot_strata_error_box.py

Rysuje boxploty błędów estymatorów stratyfikowanych.


In [ ]:
show_code("scripts/plot_strata_error_box.py")


### scripts/generate_european_estimator_replications.py

Generuje replikacje estymatorów opcji europejskich i je zapisuje.


In [ ]:
show_code("scripts/generate_european_estimator_replications.py")


### scripts/generate_barrier_estimator_replications.py

Generuje replikacje estymatorów opcji barierowych i je zapisuje.


In [ ]:
show_code("scripts/generate_barrier_estimator_replications.py")


### scripts/generate_european_estimator_stats.py

Wylicza statystyki z replikacji estymatorów opcji europejskich.


In [ ]:
show_code("scripts/generate_european_estimator_stats.py")


### scripts/generate_barrier_estimator_stats.py

Wylicza statystyki z replikacji estymatorów opcji barierowych.


In [ ]:
show_code("scripts/generate_barrier_estimator_stats.py")


### scripts/generate_european_vrf_table.py

Buduje tabele VRF dla estymatorów europejskich.


In [ ]:
show_code("scripts/generate_european_vrf_table.py")


### scripts/generate_barrier_vrf_table.py

Buduje tabele VRF dla estymatorów barierowych.


In [ ]:
show_code("scripts/generate_barrier_vrf_table.py")


### scripts/generate_european_error_plots.py

Generuje wykresy błędów estymacji dla opcji europejskich.


In [ ]:
show_code("scripts/generate_european_error_plots.py")
